In [2]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import re
import tensorflow as tf
from pathlib import Path
from keras.utils.np_utils import to_categorical

In [3]:
data = pd.read_csv( '/Volumes/Cisco/Fall2021/onnx-exchange/Training/IMDB Dataset.csv')
print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [4]:
english_stops = set(stopwords.words('english'))

In [5]:
def load_dataset():
    df = pd.read_csv('/Volumes/Cisco/Fall2021/onnx-exchange/Training/IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


In [6]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
37952    [this, really, one, star, many, clich, predict...
40832    [i, first, came, across, my, tutor, friend, ac...
10019    [possible, spoilers, although, done, better, m...
22387    [dr, paul, flanner, richard, gere, successful,...
35059    [if, seen, eva, longoria, tv, show, desperate,...
                               ...                        
33442    [this, made, tv, film, brilliant, one, this, p...
31510    [first, i, want, say, people, give, poor, revi...
18804    [of, ten, actors, portrayed, philo, vance, ser...
24603    [just, finished, watching, say, i, impressed, ...
48668    [i, christian, thought, movie, pretty, good, w...
Name: review, Length: 40000, dtype: object 

32324    [the, remarkable, fact, participation, klaus, ...
7724     [very, cliched, quite, corny, acting, gets, wo...
43791    [this, movie, groundbreaking, former, soviet, ...
33659    [ok, lets, see, what, funny, first, movie, i, ...
30726    [intense, fascinating, drama, similarities, to...
 

In [7]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [8]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[    8    15     5 ...     0     0     0]
 [    1    23   290 ... 17217  1674    75]
 [  534   977   165 ...    25  1294 18832]
 ...
 [  274   640    67 ...     0     0     0]
 [  452  1790    65 ...     0     0     0]
 [    1  1421    95 ...     0     0     0]] 

Encoded X Test
 [[    2  1723   103 ...     0     0     0]
 [  809  7461    86 ...     0     0     0]
 [    8     3  6716 ...     0     0     0]
 ...
 [    2    66  2343 ...     0     0     0]
 [ 1324  7384  3143 ...     0     0     0]
 [  624 23498    35 ...  1568   331   746]] 

Maximum review length:  131


In [9]:
import time
import os
import copy
import csv
import pandas as pd
from datetime import datetime

date = datetime.today().strftime('%Y-%m-%d')

In [10]:
from torch.utils.data import TensorDataset, DataLoader
import torch

In [11]:
import onnx
import onnxruntime
import coremltools
import time
import tf2onnx

In [12]:
onnx_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/onnx/'
coreml_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/coremltools/'
error_path = '/Volumes/Cisco/Fall2021/onnx-exchange/miss-classification/v2/'

In [13]:
def convert_category(y):
    
    #list_ = []
    #for i in y:
    #    val = 1
    #    if i < 0.5:
    #        val = 0
    #    list_.append(val)
    return np.argmax(y) #np.array(list_)
#convert_category(k_predict)

In [14]:
def model_scores(y_test, test_predict):
    correct_ = 0 #y_test = test_predict #np.sum(y_test == test_predict)
    if y_test == test_predict:
        correct_ = 1
    accuracy  = correct_#*100./np.sum(y_test == y_test)
    return accuracy
def get_miss_classification(y1, y2):
    miss_perc_val_original_coreml = 0
    if y1 != y2:
        miss_perc_val_original_coreml = 1
    return miss_perc_val_original_coreml #, precision2, recall2, f12

In [32]:
def to_onnx(i, x, y, batch_size):
    
    # Input to the model
    #device_reset = cuda.get_current_device()
    #device_reset.reset()
    #x.cuda()
    #print("converting for batch: ", i)
    y = np.argmax(y)
    #torch.random.manual_seed(42)
    #x = torch.randn(10000, 3, 32, 32, requires_grad=True)
    
    ### Original Model
    since_1 = time.time()
    #model = torch.load(path+model_name+'.pth')
    try:
        with tf.device('/cpu:0'): 
            k_predict = model.predict(x)
    except Exception as e:
        print('Error keras: ')
        return None
    inference_time_original = time.time() - since_1
    y0 = convert_category(k_predict)
    correct_original = np.sum(y0 == y)
    accuracy_original = model_scores(y, y0)
    miss_test_original = get_miss_classification(y, y0)
    # ONNX Model
    
    t_elapsed_2 = time.time() - since_1
    #since_1 = time.time()
    #onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    #load_time_onnx = time.time() - since_1
    #onnx.checker.check_model(onnx_model)
    #def to_numpy(tensor):
    #    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    #ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    since_1 = time.time()
    ort_inputs = {ort_session.get_inputs()[0].name: x}
    ort_outs = ort_session.run(None, ort_inputs)
    inference_time_onnx = time.time() - since_1
    # compare ONNX Runtime and PyTorch results
    #print("\n*********\n\n")
    #time_diff = t_elapsed_0+t_elapsed_1, t_elapsed_2, t_elapsed_3
    
    ####### Mis-classification ONNX ######################################
    y2 = convert_category(ort_outs[0])
    correct_onnx = np.sum(y2 == y)
    accuracy_onnx = model_scores(y, y2)
    miss_original_onnx = get_miss_classification(y0, y2)
    miss_test_onnx = get_miss_classification(y, y2)
    ####### End of mis-classification ONNX ###################################### 
    
    
    ## CoreML
    
    #since_1 = time.time()
    #coreml_model = coremltools.models.MLModel(coreml_path+framework+"/{}/{}.mlmodel".format(model_short_name, model_name))
    #load_time_coreml = time.time() - since_1
    
    #spec = coreml_model.get_spec()
    #coreml_model = coremltools.models.MLModel(spec)
    split_ = str(coreml_model.get_spec().description.input[0]).split('\n')
    name_1 = split_[0].replace('name: "', '')
    name_1 = name_1.replace('"', '')
    
    since_1 = time.time()
    output_dict_test = coreml_model.predict({name_1:x})
    inference_time_coreml = time.time() - since_1
    ####### Mis-classification coreML ######################################
    y3 = convert_category(output_dict_test['Identity'])
    correct_coreml = np.sum(y3 == y)
    accuracy_coreml = model_scores(y, y3)
    miss_original_coreml = get_miss_classification(y0, y3)
    miss_test_coreml = get_miss_classification(y, y3)
    
    ####### End of mis-classification coreML ######################################
    #return correct_original,correct_onnx,correct_coreml 
    list_val = [accuracy_original, accuracy_onnx, accuracy_coreml, miss_original_onnx, miss_original_coreml,inference_time_original, inference_time_onnx, inference_time_coreml]
    return list_val

In [20]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

def _lets_convert(data, data_writer_run, batch_size):
    since = time.time()
    accuracy_original = 0.0
    accuracy_onnx = 0.0
    accuracy_coreml = 0.0
    #miss_test_original = 0
    miss_original_onnx = 0
    #miss_test_onnx = 0
    miss_original_coreml = 0
    #miss_test_coreml = 0
    
    inference_time_original = 0.0
    inference_time_onnx = 0.0
    inference_time_coreml = 0.0
    
    total_ = 0.0
    total_datasets = 0
    
    for i, (inputs, labels) in enumerate(data):
        inputs = to_numpy(inputs)
        labels = to_numpy(labels)
        list_val = to_onnx(i, inputs,labels, batch_size)
        if i%50 == 0:
            print(i, list_val)
        if list_val != None:
            accuracy_original += list_val[0]
            accuracy_onnx += list_val[1]
            accuracy_coreml += list_val[2]

            miss_original_onnx += list_val[3]
            miss_original_coreml += list_val[4]

            inference_time_original += list_val[5]
            inference_time_onnx += list_val[6]
            inference_time_coreml += list_val[7]
            total_ += np.sum(labels == labels)
            total_datasets += labels.shape[0]
        if i == 1000:
            break
    time_elapsed = time.time() - since
    print('Conversion complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60) )
    #print(correct_original, correct_onnx, correct_coreml, total, correct_original*100./total, precision2, recall2, f1)
    #return correct_original*100./total, correct_onnx*100./total, correct_coreml*100./total
    data_writer_run.writerow([model_short_name,framework, training_id, model_name, round(accuracy_original*100/total_,2), round(accuracy_onnx*100/total_,2), round(accuracy_coreml*100/total_,2),  round(miss_original_onnx*100/total_datasets,2),  round(miss_original_coreml*100/total_datasets,2),'', inference_time_original/total_datasets, inference_time_onnx/total_datasets, inference_time_coreml/total_datasets, '',
                              '{:.0f}m {:.0f}s'.format((inference_time_original/total_datasets) // 60, (inference_time_original/total_datasets) % 60),
                              '{:.0f}m {:.0f}s'.format((inference_time_onnx/total_datasets) // 60, (inference_time_onnx/total_datasets) % 60),
                              '{:.0f}m {:.0f}s'.format((inference_time_coreml/total_datasets) // 60, (inference_time_coreml/total_datasets) % 60),
                              '{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)])

In [35]:
import pandas as pd 

#data_file_run = open(error_path+framework+"/{}/runtime_miss-classification_{}.csv".format(model_short_name,model_name), mode='w', newline='',
#                                  encoding='utf-8')
#data_writer_run = csv.writer(data_file_run, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#data_writer_run.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','runtime','original_load_time', 'original_infererence_time', 'runtime_load_time', 
#                          'runtime_inference_time',  'miss_classified_original_runtime_percentage','',  'encoded_miss_classified_original_runtime_percentage','encoded_miss_classified_original_test_runtime_percentage', '', 'accuracy_original', 'accuracy_runtime'])

for round_ in [3,4,5,6,7,8,9,10]: #,3,4,5,6,7,8,9,10
    training_id = round_
    model_short_name = 'lstm'
    framework = 'keras'
    path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/{}/{}/'.format(framework, model_short_name)
    since_0 = time.time()
    #model_path = 'tf_Lenet5_mnist_2021-08-24-10:35:35'
    #model_name = 'tf_alexnet_cifar10_2021-08-27-17:05:27'
    #model_name = 'tf_resnet18-cifar10_2021-10-29_{}'.format(training_id)
    model_name = 'tf_lstm-imdb_2021-10-30_{}'.format(training_id)
    #model_name = 'tf_gru-imdb_2021-10-29_{}'.format(training_id)
    model = tf.keras.models.load_model(path+ model_name+'.h5')
    t_elapsed_0 = time.time() - since_0
    size0 = os.path.getsize(path+ model_name+'.h5')
    size0
    
    since_1 = time.time()
    onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    load_time_onnx = time.time() - since_1
    onnx.checker.check_model(onnx_model)
    ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    
    
    since_1 = time.time()
    coreml_model = coremltools.models.MLModel(coreml_path+framework+"/{}/{}.mlmodel".format(model_short_name, model_name))
    load_time_coreml = time.time() - since_1
    
    if not os.path.exists(error_path+framework):
        Path(error_path+framework).mkdir(parents=True, exist_ok=True)
    
    print('round: ', round_)
    flag = 0
    if not os.path.exists(error_path+framework+"/runtime_accuracy_{}.csv".format(model_short_name)):
        data_file_run2 = open(error_path+framework+"/runtime_accuracy_{}.csv".format(model_short_name), mode='w', newline='', encoding='utf-8')
    else:
        data_file_run2 = open(error_path+framework+"/runtime_accuracy_{}.csv".format(model_short_name), mode='a+', newline='', encoding='utf-8')
        flag = 1
    data_writer_run2 = csv.writer(data_file_run2, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    if flag == 0: 
        data_writer_run2.writerow(['model','framework', 'training_id', 'model_full', 'accuracy_original', 'accuracy_onnx', 'accuracy_coreml', 'miss_original_onnx', 'miss_original_coreml','', 'inference_time_original', 'inference_time_onnx', 'inference_time_coreml', '', 'inference_time_original2', 'inference_time_onnx2', 'inference_time_coreml2', 'overral_time'])
        
    #train_data = TensorDataset(torch.as_tensor(np.array(x_train).astype('int32')), torch.as_tensor(np.array(y_train).astype('int32')))
    valid_data = TensorDataset(torch.as_tensor(np.array(x_test).astype('int32')), torch.as_tensor(np.array(y_test).astype('int32')))

    # dataloaders
    batch_size = 1 

    # make sure to SHUFFLE your data
    #train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
    _lets_convert(valid_loader, data_writer_run2, batch_size)
    data_file_run2.close()

round:  3
0 [1, 1, 1, 0, 0, 4.074235916137695, 0.05910301208496094, 0.09341907501220703]
50 [1, 1, 1, 0, 0, 0.0867609977722168, 0.03722810745239258, 0.03632187843322754]
100 [1, 1, 1, 0, 0, 0.09136605262756348, 0.039283037185668945, 0.02864694595336914]
150 [1, 1, 1, 0, 0, 3.508718967437744, 0.03668403625488281, 0.028325796127319336]
200 [1, 1, 1, 0, 0, 0.09247303009033203, 0.03982210159301758, 0.04764389991760254]
250 [1, 1, 1, 0, 0, 0.11222195625305176, 0.04260897636413574, 0.031039953231811523]
300 [1, 1, 1, 0, 0, 0.11060190200805664, 0.04209113121032715, 0.029939889907836914]
350 [1, 1, 1, 0, 0, 0.09514307975769043, 0.03307294845581055, 0.025784015655517578]
400 [1, 1, 1, 0, 0, 0.08807182312011719, 0.039130210876464844, 0.02730584144592285]
450 [1, 1, 1, 0, 0, 0.10570788383483887, 0.04256296157836914, 0.032002925872802734]
500 [1, 1, 1, 0, 0, 0.08911609649658203, 0.04902911186218262, 0.039801836013793945]
550 [1, 1, 1, 0, 0, 0.08852505683898926, 0.03406524658203125, 0.0277040004730

KeyboardInterrupt: 

In [47]:
training_id = 4
model_short_name = 'lstm'
framework = 'keras'
path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/{}/{}/'.format(framework, model_short_name)
since_0 = time.time()
#model_path = 'tf_Lenet5_mnist_2021-08-24-10:35:35'
#model_name = 'tf_alexnet_cifar10_2021-08-27-17:05:27'
#model_name = 'tf_resnet18-cifar10_2021-10-29_{}'.format(training_id)
model_name = 'tf_lstm-imdb_2021-10-30_{}'.format(training_id)
#model_name = 'tf_gru-imdb_2021-10-29_{}'.format(training_id)
model = tf.keras.models.load_model(path+ model_name+'.h5')

In [15]:
path = '/Volumes/Cisco/Summer2022/onnx-exchange/Train2/trial/keras/lstm/'
since_0 = time.time()
#model_path = 'tf_Lenet5_mnist_2021-08-24-10:35:35'
#model_name = 'tf_alexnet_cifar10_2021-08-27-17:05:27'
#model_name = 'tf_resnet18-cifar10_2021-10-29_{}'.format(training_id)
model_name = 'amin-lstm'
#model_name = 'tf_gru-imdb_2021-10-29_{}'.format(training_id)
model = tf.keras.models.load_model(path+ model_name+'.h5')
t_elapsed_0 = time.time() - since_0
size0 = os.path.getsize(path+ model_name+'.h5')
size0

32334384

In [17]:
batch_size = 128
valid_data = TensorDataset(torch.as_tensor(np.array(x_test).astype('int32')), torch.as_tensor(np.array(y_test).astype('int32')))

valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)

In [29]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
max_features = 20000
maxlen = 80
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [48]:
#model = tf.keras.models.load_model(path+ model_name+'.h5')
total_ = 0
accuracy_original = 0
k_predict = model.predict(x_test)

k_predict_list = []
for val in k_predict:
    if val < 0.5:
        k_predict_list.append(0)
    else:
        k_predict_list.append(1)
k_predict_list = np.array(k_predict_list)
correct = np.sum(y_test == k_predict_list)
total_ = np.sum(y_test == y_test)
print(round(correct*100/total_,2))

50.34


In [41]:
y_test

array([0, 1, 1, ..., 0, 0, 0])

In [42]:
k_predict_list

array([0, 1, 1, ..., 0, 0, 1])

In [45]:
correct = np.sum(y_test == k_predict_list)
total_ = np.sum(y_test == y_test)
print(round(correct*100/total_,2))

81.96


In [38]:
#y0 = convert_category(k_predict)
accuracy_original += model_scores(y_test, k_predict_list)
total_ += np.sum(y_test == y_test)
    
print(round(accuracy_original*100/total_,2))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [49]:
np.testing.assert_array_equal(y_test, k_predict_list)

AssertionError: 
Arrays are not equal

Mismatched elements: 12415 / 25000 (49.7%)
Max absolute difference: 1
Max relative difference: 1.
 x: array([0, 1, 1, ..., 0, 0, 0])
 y: array([1, 1, 1, ..., 1, 1, 1])

In [51]:
try:
    np.testing.assert_array_equal(y_test, k_predict_list)
except Exception as e:
    #print(e)
    for line_ in str(e).split('\n'):
        #print(' ---- : ', line_)
        if 'Mismatched elements' in line_:
            value = line_.replace('Mismatched elements: ', '').strip()
            val = value.split('(')[0].split('/')[0].replace(' ', '')
            print(val)
            #miss_perc_val_test_runtime = value[value.find("(")+1:value.find(")")]
            #print(value, perc_val)
            break

12415
